Define 2 classes - OwnerInfo (which represents information about the inventory owner) and Inventory (which represents each item in the inventory data).

In OwnerInfo, initialise the following attributes: owner_name, owner_address, owner_telephone.

In Inventory, initialise the following attributes: purchase_date, serial_number, description, source_style_area, value.

In [22]:
# the data models 
class OwnerInfo:
    def __init__(self, owner_name, owner_address, owner_telephone):
        self.owner_name = owner_name, 
        self.owner_address = owner_address
        self.owner_telephone = owner_telephone

class Inventory: 
    def __init__(
            self, 
            purchase_date, 
            serial_number, 
            description, 
            source_style, 
            value
    ): 
        self.purchase_date = purchase_date
        self.serial_number = serial_number
        self.description = description
        self.source_style = source_style
        self.value = value

# instantiate ownerIngo and Inventory 
inventory_1 = Inventory(
    "3-04-2025", 
    "4535", 
    "a nce one", 
    "luxury", 
    "790"
)
owner_1 = OwnerInfo("Oladimeji", "FUTA North gate", "45324")
print(type(inventory_1))
print(type(owner_1))



<class '__main__.Inventory'>
<class '__main__.OwnerInfo'>


Create a function get_data_from_pdf that reads a pdf from path and uses a relevant parser package to extract the content of the pdf. The output of this function should be the raw text extracted from the pdf.


Create a function align_content that processes the raw text from the preceding process and aligns them line by line as it appears in the PDF.

Create a function extract_data that processes the aligned content to create the final response JSON. In this function you will manipulate the text to extract key information and assign them to the OwnerInfo class and Inventory class attributes respectively.
In extract_data, create a dictionary that stores the extracted OwnerInfo attributes as first-level objects and the Inventory attributes as second-level objects nested within a "data" object which will be on the same level with the OwnerInfo keys. The "data" object should be a list of dicts, where each dict represents one inventory item.

In [23]:
import os
os.chdir("/home/boladimeji/Desktop/CRR/data-extraction-pipeline")

print("Now working from:", os.getcwd())

Now working from: /home/boladimeji/Desktop/CRR/data-extraction-pipeline


In [24]:
import pdfplumber

with pdfplumber.open("data/home_inventory.pdf") as pdf: 
    text = ""
    for page in pdf.pages: 
        text = text + page.extract_text()
print(text)


def get_data_from_pdf(path): 
    with pdfplumber.open(path) as pdf: 
        text = ""
        for page in pdf.pages: 
            text = text + page.extract_text()
    return text

text = get_data_from_pdf("data/home_inventory.pdf")


HOME INVENTORY
Owner Information
Name John Doe
Address 123 Maple Street
City, Zip, Address Portland, OR 97203
Phone Number (503) 555-1234
S/N Area Item Description Source Purchase Date Style Serial No Value
1 Living Room Desk Target 07/06/2018 Premium 6DDZ7S36 $ 846.59
2 Kitchen LED TV Walmart 31/05/2015 Classic NJEZ3OPO $ 382.04
3 Living Room LED TV Target 03/03/2019 Premium HRIS4LI8 $ 1,603.37
4 Garage Dining Table Wayfair 26/03/2023 Modern 2HLNMD64 $ 552.74
5 Living Room Tool Set Target 06/01/2020 Classic R08QDU0S $ 1,546.39
6 Office LED TV Amazon 03/04/2015 Classic GSABG41R $ 1,319.36
7 Office Mattress Amazon 28/02/2022 Premium DVTMS64O $ 573.10
8 Bedroom Tool Set Home Depot 07/04/2017 Classic RB93WPTH $ 1,421.38
9 Dining Room Mattress Target 20/03/2017 Compact 7RX8YNW9 $ 1,760.11
10 Living Room Desk Target 30/06/2019 Classic US1B0BQI $ 941.43
11 Garage Dining Table Wayfair 09/02/2016 Compact 1J7088BO $ 71.12
12 Bedroom Desk Home Depot 21/08/2025 Modern GVJFQYT1 $ 841.41
13 Office 

In [25]:
for line in text: 
    line.strip()

    if not line:
        continue
def align_content(text): 
    for line in text: 
        line.strip()
        continue
    return text
text

'HOME INVENTORY\nOwner Information\nName John Doe\nAddress 123 Maple Street\nCity, Zip, Address Portland, OR 97203\nPhone Number (503) 555-1234\nS/N Area Item Description Source Purchase Date Style Serial No Value\n1 Living Room Desk Target 07/06/2018 Premium 6DDZ7S36 $ 846.59\n2 Kitchen LED TV Walmart 31/05/2015 Classic NJEZ3OPO $ 382.04\n3 Living Room LED TV Target 03/03/2019 Premium HRIS4LI8 $ 1,603.37\n4 Garage Dining Table Wayfair 26/03/2023 Modern 2HLNMD64 $ 552.74\n5 Living Room Tool Set Target 06/01/2020 Classic R08QDU0S $ 1,546.39\n6 Office LED TV Amazon 03/04/2015 Classic GSABG41R $ 1,319.36\n7 Office Mattress Amazon 28/02/2022 Premium DVTMS64O $ 573.10\n8 Bedroom Tool Set Home Depot 07/04/2017 Classic RB93WPTH $ 1,421.38\n9 Dining Room Mattress Target 20/03/2017 Compact 7RX8YNW9 $ 1,760.11\n10 Living Room Desk Target 30/06/2019 Classic US1B0BQI $ 941.43\n11 Garage Dining Table Wayfair 09/02/2016 Compact 1J7088BO $ 71.12\n12 Bedroom Desk Home Depot 21/08/2025 Modern GVJFQYT1 

In [26]:
import re
import json

owner_info_pattern = (
    r"Name\s+(?P<name>.+?)\n"
    r"Address\s+(?P<address>.+?)\n"
    r'City, Zip, Address\s*(?P<city_zip>.+?)\s*'
    r'Phone Number\s*(?P<phone>\(\d{3}\)\s*\d{3}-\d{4})'
)


inventory_pattern = re.compile(r"""
    (?P<no>\d+)\s+                            
    (?P<area>[A-Za-z ]+)\s+                  
    (?P<item_description>[A-Za-z ]+)\s+       
    (?P<source>[A-Za-z ]+)\s+                  
    (?P<purchase_date>\d{2}/\d{2}/\d{4})\s+    
    (?P<style>[A-Za-z]+)\s+                   
    (?P<serial_no>[A-Za-z0-9]+)\s+    
    \$\s*(?P<value>[0-9,]+\.\d{2})        
    """, 
    re.VERBOSE
)





In [27]:
owner_match = re.search(owner_info_pattern, text, re.DOTALL)
owner_details = list(owner_match.groups())
owner_details

['John Doe', '123 Maple Street', 'Portland, OR 97203', '(503) 555-1234']

In [28]:
inventory_items = []
for match in inventory_pattern.finditer(text):
    inventory_item = match.groupdict()
    
    inventory_items.append(inventory_item)

    # print(inventory_item)



In [29]:
inventory_items

[{'no': '1',
  'area': 'Living Room',
  'item_description': 'Desk',
  'source': 'Target',
  'purchase_date': '07/06/2018',
  'style': 'Premium',
  'serial_no': '6DDZ7S36',
  'value': '846.59'},
 {'no': '2',
  'area': 'Kitchen LED',
  'item_description': 'TV',
  'source': 'Walmart',
  'purchase_date': '31/05/2015',
  'style': 'Classic',
  'serial_no': 'NJEZ3OPO',
  'value': '382.04'},
 {'no': '3',
  'area': 'Living Room LED',
  'item_description': 'TV',
  'source': 'Target',
  'purchase_date': '03/03/2019',
  'style': 'Premium',
  'serial_no': 'HRIS4LI8',
  'value': '1,603.37'},
 {'no': '4',
  'area': 'Garage Dining',
  'item_description': 'Table',
  'source': 'Wayfair',
  'purchase_date': '26/03/2023',
  'style': 'Modern',
  'serial_no': '2HLNMD64',
  'value': '552.74'},
 {'no': '5',
  'area': 'Living Room Tool',
  'item_description': 'Set',
  'source': 'Target',
  'purchase_date': '06/01/2020',
  'style': 'Classic',
  'serial_no': 'R08QDU0S',
  'value': '1,546.39'},
 {'no': '6',
  'ar

Create a function extract_data that processes the aligned content to create the final response JSON. In this function you will manipulate the text to extract key information and assign them to the OwnerInfo class and Inventory class attributes respectively.
In extract_data, create a dictionary that stores the extracted OwnerInfo attributes as first-level objects and the Inventory attributes as second-level objects nested within a "data" object which will be on the same level with the OwnerInfo keys. The "data" object should be a list of dicts, where each dict represents one inventory item.

In [30]:
os.chdir("/home/boladimeji/Desktop/CRR/data-extraction-pipeline")

In [31]:
from src.data_models import OwnerInfo, Inventory
from datetime import datetime

In [41]:
def extract_data():
    owner_info_pattern = (
    r"Name\s+(?P<name>.+?)\n"
    r"Address\s+(?P<address>.+?)\n"
    r'City, Zip, Address\s*(?P<city_zip>.+?)\s*'
    r'Phone Number\s*(?P<phone>\(\d{3}\)\s*\d{3}-\d{4})'
    )


    inventory_pattern = re.compile(r"""
        (?P<no>\d+)\s+                            
        (?P<area>[A-Za-z ]+)\s+                  
        (?P<item_description>[A-Za-z ]+)\s+       
        (?P<source>[A-Za-z ]+)\s+                  
        (?P<purchase_date>\d{2}/\d{2}/\d{4})\s+    
        (?P<style>[A-Za-z]+)\s+                   
        (?P<serial_no>[A-Za-z0-9]+)\s+    
        \$\s*(?P<value>[0-9,]+\.\d{2})        
        """, 
        re.VERBOSE
    )

    owner_match = re.search(owner_info_pattern, text, re.DOTALL)
    owner_details = list(owner_match.groups())

    owner = OwnerInfo(
        owner_name=owner_details[0], 
        owner_address=owner_details[1], 
        owner_telephone=owner_details[3]
    )



    inventories = []
    for match in inventory_pattern.finditer(text):
        inventory = match.groupdict()

        dt = datetime.strptime(inventory['purchase_date'], "%d/%m/%Y")
        inventory_object = Inventory(
            purchase_date=dt.isoformat(), 
            serial_number=inventory["serial_no"], 
            description=inventory["item_description"], 
            source_style=inventory["style"], 
            value=inventory["value"]
        )

        inventories.append(inventory_object)
        
    return {
        **owner.__dict__, 
        "data": [inventory.__dict__ for inventory in inventories]
    }
    

In [42]:
inventory = extract_data()
with open("inventory.json", "w") as f: 
    json.dump(inventory, f, indent=4)

In [34]:
!pip install pdfplumber